In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import pandas as pd
from matplotlib import pyplot as plt
from datetime import date

In [ ]:
# Load our CSV file into a dataframe.
time_entries = pd.read_csv(
    "toggl.csv", 
    parse_dates=["Start date", "End date"],
    date_format="%Y-%m-%d",
)
print(f'Data has {time_entries.shape[0]} rows and {time_entries.shape[1]} columns')
time_entries.head()

In [ ]:
# Drop unneeded columns.
time_entries = time_entries.drop(['User', 'Email', 'Client', 'Task', 'Billable', 'Tags', 'Amount ()'], axis=1)

# Toggl provides the data with dates and times separated. We need to merge them back together.
# First, we convert the "Start time" column to `pandas.dt.time`. Then we use `pd.Timestamp.combine()`
# to combine the date and time columns into a new column. Finally, we remove the original columns.
time_entries['Start time'] = pd.to_datetime(time_entries['Start time'], format="%H:%M:%S").dt.time
time_entries['Start'] = time_entries.apply(lambda x: pd.Timestamp.combine(x['Start date'], x['Start time']), axis=1)
time_entries['End time'] = pd.to_datetime(time_entries['End time'], format="%H:%M:%S").dt.time
time_entries['End'] = time_entries.apply(lambda x: pd.Timestamp.combine(x['End date'], x['End time']), axis=1)
time_entries = time_entries.drop(['Start date', 'Start time', 'End date', 'End time'], axis=1)

# Recalculate the "Duration" column so that we get instances of `pandas.Timedelta`.
time_entries['Duration'] = time_entries['End'] - time_entries['Start']

time_entries.head()

In [ ]:
# Filter data by date (`start_date` inclusive, `end_date` exclusive).
# TODO: move this up before we perform the processing.
start_date = date(2021, 10, 1)
end_date = date(2022, 11, 1)
time_entries = time_entries[(time_entries['Start'].dt.date >= start_date) & (time_entries['End'].dt.date < end_date)]
print(f'Data has {time_entries.shape[0]} rows after filtering.')
time_entries.head()

In [ ]:
# Calculate the time spent per project.
time_spent = time_entries.groupby('Project')['Duration'].sum().dt.total_seconds() / 3600
time_spent.sort_values(inplace=True)

ax = time_spent.plot.barh()
ax.set_title('Time spent by project')
ax.set_xlabel('Hours')
ax.set_ylabel('')

In [ ]:
# TODO: Markdown table with basic statistical analysis. (And move above the plot).

In [ ]:
# What do we want:
# - Time per weekday
# - Time per day (weekend)

In [ ]:
# Bucket per week. https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html
per_week = time_entries.groupby(pd.Grouper(key="Start", freq="1W"))['Duration'].sum().dt.total_seconds() / 3600
ax = per_week.plot.bar()
ax.set_title('Recorded time, per week')
ax.set_xlabel('Date')
ax.set_ylabel('Hours')
# TODO: set ticks per month
ax.xaxis.set_major_locator(plt.MaxNLocator(6))

In [ ]:
per_day = time_entries.groupby(time_entries['Start'].dt.weekday)['Duration'].sum().dt.total_seconds() / 3600
ax = per_day.plot.bar()
ax.set_title('Recorded time, by day of week')
ax.set_xlabel('Day of Week')
ax.set_ylabel('Hours recorded')

In [ ]:
# Add a column that assigns a unique year-week string.
# time_entries['week'] = time_entries.apply(lambda x: str(x['Start'].year) + '-' + str(x['Start'].week), axis=1)
time_entries['week_start'] = time_entries['Start'].dt.to_period('W').apply(lambda r: r.start_time)
# Faster: df['week_start'] = df['myday'] - pd.to_timedelta(df['myday'].dt.weekday, unit='D'). 
per_week = time_entries.groupby(['Project', 'week_start'])['Duration'].sum().dt.total_seconds() / 3600


import matplotlib.pyplot as plt
fig, ax = plt.subplots()
# ax.bar(per_week['AI'].index, per_week['AI'].values)
# ax.bar(per_week['Sport'].index, per_week['Sport'].values)

# TODO: we are missing a few days here. start_date should be aligned to a Monday

# Create dates aligned on Monday.
dates = pd.date_range(start_date, end_date, freq='W-MON')
running_sum = [0 for _ in dates]

projects = set(index[0] for index in per_week.index.values)

for project in projects:
    values = [0] * len(dates)
    for i, d in enumerate(dates):
        values[i] = running_sum[i] + per_week[project].get(str(d.date()), 0)
    ax.bar(dates, values, bottom=running_sum, label=project)
    running_sum = values

ax.set_title('Time tracked, per week')
ax.legend()